<a href="https://colab.research.google.com/github/fahmizainal17/Machine-Learning-Project-to-predict-salary-/blob/main/PredictSalary_Model_Object.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Import Libraries and Packages

Load the Model

Steps :
1. Copy path of invoke_xgbc_model.pkl and paste it at import pickle snippet.

2. Copy path of cleaned_dataTransformed and paste it at import data snippet.

3. Run all.

Note that
     


In [27]:
import pickle

# Load the saved pipeline and model
with open('/content/xgbc_model.pkl', 'rb') as file:
    loaded_pipeline = pickle.load(file)


In [28]:
# Required
import numpy as np
import pandas as pd

# Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import FunctionTransformer

# Import models
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

# Metrics
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.model_selection import cross_val_score, cross_validate, StratifiedKFold
from sklearn.metrics import roc_auc_score

# 2. Define predictSalary function

In [29]:
# Define function predictSalary
def predictSalary(model, data):
    X = data.drop("salary", axis=1) # Features/predictor
    Y = data["salary"] # Target

    # Split dataset
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.10, random_state=1) # 90:10 (Train:Test)

    # Train the model
    model.fit(x_train, y_train) # Model will try to learn

    # Predict probabilities and labels
    prediction_prob = model.predict_proba(x_test) # Probability of a class
    prediction = model.predict(x_test) # Exact prediction

    # Calculate accuracy, precision, recall, F1-score, log loss
    test_accuracy = model.score(x_test, y_test)
    train_accuracy = model.score(x_train, y_train)
    precision = precision_score(y_test, prediction)
    recall = recall_score(y_test, prediction)
    f1 = f1_score(y_test, prediction)

    # Cross Validation
    # Stratified sampling instead of random sampling for equal proportion
    stratified_kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

    # Perform cross-validation on the test data and calculate mean accuracy and log loss
    results = cross_validate(model,
                              X,
                              Y,
                              cv=stratified_kf,
                              scoring='roc_auc')

    # Extract the scores
    auc_scores = results['test_score']

    # Calculate the means for each cross validated evaluation metrics
    mean_auc_scores = np.mean(auc_scores)

    # Print the evaluation metrics
    print("*********Check Overfitting/Underfitting*********")
    print(f"Train Accuracy: {train_accuracy:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f}")
    print("*********Evaluating Metric*********")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-score: {f1:.4f}")
    print(f"Mean AUC: {mean_auc_scores:.4f}")
    print("**************************")

    return mean_auc_scores

# 3. Import data

In [30]:
# Define data
df = pd.read_csv('/content/cleaned_dataTransformed.csv')

# 4. Define Model Pipeline

In [31]:
# Define pipeline
right_skew_numerical_transform = ['house_rental_fee',
                                  'house_loan_pmt',
                                  'transport_spending',
                                  'public_transport_spending',
                                  'house_utility',
                                  'food_spending',
                                  'kids_spending',
                                  'personal_loan',
                                  'education_loan',
                                  'other_loan',
                                  'investment']

# Right Skew Pipeline
preprocessing_pipeline1 = Pipeline([
    ('log1p_transform', FunctionTransformer(func=np.log1p, validate=False)), # Convert from right skew to symmetrical
    ('robust_scaling', RobustScaler()), # Removes outliers
    ('standard_scaling', StandardScaler()) # Standardization (create similar scale between features)
])

# Create a column transformer using the pipelines for skew and categorical features
transformer = ColumnTransformer(
    transformers=[
        ('Right Skew Num', preprocessing_pipeline1, right_skew_numerical_transform),
    ],
    remainder="passthrough"
)

# 4. Define Model Object

In [32]:
# Define model object with best parameters obtained during the model building
model = loaded_pipeline

# 5. Predict!

In [33]:
mean_auc_scores = predictSalary(model, df)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [20:59:44] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "max_features", "max_leaf_nodes" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [20:59:44] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "max_features", "max_leaf_nodes" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [20:59:44] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "max_features", "max_leaf_nodes" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [20:59:44] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "max_features", "max_leaf_nodes" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [20:59:45] WARNING: /works

*********Check Overfitting/Underfitting*********
Train Accuracy: 0.9440
Test Accuracy: 0.9078
*********Evaluating Metric*********
Precision: 0.6538
Recall: 0.6071
F1-score: 0.6296
Mean AUC: 0.9297
**************************


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [20:59:45] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "max_features", "max_leaf_nodes" } are not used.

  warnings.warn(smsg, UserWarning)
